# Advanced Tallies in OpenMC

In this tutorial, we'll learn about more advanced tally options.

A model that warrants advanced tallies is necessarily more complex than a pincell, so we're going to use the built-in PWR assembly model in OpenMC.

In [ ]:
model.geometry.root_universe.plot(width=(22, 22), pixels=(600, 600))

In order to make some of the plots we'll generate in this session more intuitive, we'll remove the reflecting boundary condition for the upper `YPlane` and right `XPlane` in order to introduce some asymmetry.

# Mesh Tallies

OpenMC can tally results onto regular, rectilinear, cylindrical, spherical, and unstructured meshes. Here we'll look at how to setup a regular mesh tally and visualize it for this assembly model. To do so, we need to create a mesh filter.

Learning from our last session on tallies, we'll include a tally with all of the scores needed for determining the neutron source.

With these tallies setup, we'll apply them and and run the model.

In [ ]:


model.settings.particles = 1000
model.settings.batches = 15 
model.settings.inactive = 10

In [ ]:
import matplotlib.pyplot as plt
img = plt.imshow(mesh_flux, origin='lower', extent=[-10.71, 10.71, -10.71, 10.71])
plt.xlabel('X [cm]')
plt.ylabel('Y [cm]')
plt.colorbar(img, label='Flux (unnormalized)')

Just like in our last tutorial, we need to renormalize these flux values by (i) multiplying by the neutron source rate and (ii) dividing by the volume of each tally bin in order to get into units of neutrons/cm$^2$/s.

For the volume normalization, we'll divide the flux values by the volume of a mesh voxel. Again, we're working with a 2D model so we'll assume an axial length of 1 cm.

In [ ]:
import numpy as np


In [ ]:
img = plt.imshow(mesh_flux * neutron_source / volume, origin='lower', extent=[-10.71, 10.71, -10.71, 10.71])
plt.xlabel('X [cm]')
plt.ylabel('Y [cm]')
plt.colorbar(img, label='Flux (unnormalized)')

In [ ]:
img = plt.imshow(mesh_heat, origin='lower', extent=[-10.71, 10.71, -10.71, 10.71])
plt.xlabel('X')
plt.ylabel('Y')
plt.colorbar(img, label='Heating (W/cm$^3$/s)')

### Manipulating the tally arrays

The `get_values()` method gives us an array with three dimensions: (filters, nuclides, scores). If you have multiple filters in a tally, the `get_reshaped_data()` method will give you a separate dimension for each filter. For our mesh case, this effectively gives the same thing as `get_values()` since there's only a single filter:

However, there is also an `expand_dims` argument that will expand a mesh filter into multiple dimensions:

Now we can index the array if we want to pull out specific values.

However, this will still have our two different scores one after the other. It would be easier to postprocess this data if we could extract out each score one at a time. To do so, we'll use the `get_slice` method.

## Distributed cells (distribcells)

So this gives us a fairly good idea of what the flux and power distributions look like in this model, but we often want to know the per-pin power generation rate -- something that is hard to post-process with the tallies above (especially because the mesh tally is not conformal to the geometry). We can use a distribcell tally to produce this information easily.

A distributed cell (distribcell) is how OpenMC stores cells in universes which are repeated in lattices. In short, each cell in OpenMC is associated with an *id* and an *instance*. If a cell is repeated multiple times throughout a geometry, that cell has the same id, but with unique instances.

First, we'll want to create a distribcell tally for the cell containing our fuel material. Based on the list above, our fuel material has the name "Fuel". We'll use that to identify the cell we want to setup a distribcell tally for.

As before, we'll create a normalized mean for the heating tally. A little inspection of the cell allows us to calculated the appropriate volume for the tally.